In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost as xgb
import re
import nltk
import string
from nltk.tokenize import TweetTokenizer
import unidecode

import warnings
warnings.filterwarnings('ignore')

stopwords = nltk.corpus.stopwords.words('english')

In [3]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [2]:
def remove_punctuation(word):        
    clean_word = ''.join([char for char in word if char not in string.punctuation])
    return clean_word

In [4]:
def cleaning_text(text):
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    text_tokenize = tokenizer.tokenize(text)
    wordlist = []
    for word in text_tokenize:
        word = word.lower()
        word = re.sub('(?P<url>https?://[^\s]+)', ' ', word)
        word = remove_punctuation(word)
        word = re.sub(r'[^\w]', ' ', word)
        word = unidecode.unidecode(word)
        word = re.sub(r'[0-9]','', word)
        if((word != '')&(word != ' ')&(word not in stopwords)):
            wordlist.append(word)
    clean_text = ' '.join(wordlist)
    return clean_text

In [5]:
train["clean_text"] = train["text"].apply(lambda x: cleaning_text(x))
train.rename(columns={"target":"target_label"}, inplace=True)
train.rename(columns={"location":"location_original"}, inplace=True)
train.rename(columns={"id":"id_original"}, inplace=True)
train.rename(columns={"text":"text_original"}, inplace=True)
train.rename(columns={"keyword":"keyword_original"}, inplace=True)
train.head()

,id_original,keyword_original,location_original,text_original,target_label,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified officer...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders cal...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfires pou...


In [7]:
test["clean_text"] = test["text"].apply(lambda x: cleaning_text(x))
test.rename(columns={"location":"location_original"}, inplace=True)
test.rename(columns={"id":"id_original"}, inplace=True)
test.rename(columns={"text":"text_original"}, inplace=True)
test.rename(columns={"keyword":"keyword_original"}, inplace=True)
test.head()

,id_original,keyword_original,location_original,text_original,clean_text
0,0,NaN,NaN,Just happened a terrible car crash,happened terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard earthquake different cities stay safe ev...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond geese fleeing across str...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills china taiwan


In [8]:
vectorizer = TfidfVectorizer(max_features=5000)
train_tfidf = vectorizer.fit_transform(train["clean_text"])
test_tfidf = vectorizer.transform(test["clean_text"])

In [9]:
feature_words = vectorizer.get_feature_names()
len(feature_words)

5000

In [10]:
X = train_tfidf
y = train["target_label"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [12]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5709, 5000)
(1904, 5000)
(5709,)
(1904,)


#### Ajustando hiper-parametros (usando 748 feature-words):
    - n_estimators=50, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=1, alpha=0
    SCORE: 0.761029
    - n_estimators=50, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=1, alpha=0
    SCORE: 0.767332
    - n_estimators=100, max_depth=7, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=1, alpha=0
    SCORE: 0.769958
    - n_estimators=100, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=1, alpha=0
    SCORE: 0.773634
    - n_estimators=100, max_depth=15, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=1, alpha=0
    SCORE: 0.773786
    - n_estimators=100, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=0.1, alpha=0
    SCORE: 0.777311
    - n_estimators=100, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.3, lambda=0.1, alpha=0
    SCORE: 0.775210
    - n_estimators=200, max_depth=9, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=0.5, alpha=0
    SCORE: 0.777836
    - n_estimators=200, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=0.1, alpha=0
    SCORE: 0.783088
    - n_estimators=200, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.7, lambda=0.1, alpha=0
    SCORE: 0.787815
    - n_estimators=200, max_depth=15, learning_rate=0.1, subsample=1, colsample_bytree=0.5, lambda=0.1, alpha=0
    SCORE: 0.785189
    - n_estimators=250, max_depth=13 u 11, learning_rate=0.1, subsample=1, colsample_bytree=0.7, lambda=0.1, alpha=0
    SCORE: 0.790966
    - n_estimators=250, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.7, lambda=0.1, alpha=0.1
    SCORE: 0.789391

In [24]:
model = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', max_depth=11, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.7, reg_lambda=0.1, n_jobs=1)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=250, n_jobs=1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=0.1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [25]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.794118


In [52]:
model_xgb.score(X_test, y_test)*100

78.93907563025209

In [53]:
#usando kfold y stratified kfold
model_xgb = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', max_depth=11, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, reg_lambda=0.1, n_jobs=1)

In [58]:
kfold = KFold(n_splits=6, random_state=100)
resultados = cross_val_score(model_xgb, X_train, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()*100))

Accuracy: 76.773785


In [59]:
kfold = StratifiedKFold(n_splits=8, random_state=134)
resultados = cross_val_score(model_xgb, X_train, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()*100))

Accuracy: 76.738903


In [27]:
X_test = test_tfidf

In [28]:
model = xgb.XGBClassifier(n_estimators=250, objective='binary:logistic', max_depth=11, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.7, reg_lambda=0.1, n_jobs=1)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=250, n_jobs=1, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=0.1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [29]:
y_pred = model.predict(X_test)
y_pred

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

In [30]:
test["target"] = y_pred
test[["id_original","target"]]

,id_original,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [31]:
test[["id_original","target"]].rename(columns={"id_original":"id"}).to_csv("../data/pred4_XGB_Tfidf", index=False)